In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import model_from_json, clone_model

from data_handler_CMAPS import CMAPSDataHandler
import custom_scores
import CMAPSAuxFunctions
from tunable_model import SequenceTunableModelRegression
from keras import regularizers

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

Using TensorFlow backend.


# Define different the model to test

In [2]:
def RULmodel_SN_1(input_shape):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l1_l2(l1=l1_lambda_regularization, l2=l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l1_l2(l1=l1_lambda_regularization, l2=l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [3]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    #optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN_1(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

# Create data Handler

In [4]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Run experiments

In [5]:
#datasets = [1, 2, 3, 4]
datasets = [2]
iterations = 10

window_sizes = {1:24,2:17,3:24,4:17}
strides = {1:1,2:1,3:1,4:1}
max_ruls = {1:129, 2:139, 3:129, 4:139}

scores ={1:[], 2:[], 3:[], 4:[]}
models = {1:RULmodel_SN_1}


#Create tunable model

num_features = len(selected_features)

shape = len(selected_features)*window_size
#file = open("ResultsBestModelAllDatasetsSingleSet.csv", "w")
#csvfile = csv.writer(file, lineterminator='\n')

tempScoresRMSE = []
tempScoresRHS = []
tempTime = []

#Run experiments
for i in datasets:
    
    min_rmse = 1000
    bestModel = None
    bestIndex = 0
    
    print("Working on dataset " + str(i))
        
    tempScoresRMSE = np.zeros((iterations,1))
    tempScoresRHS = np.zeros((iterations,1))
    tempTime = np.zeros((iterations,1))
        
    input_shape = window_sizes[i]*num_features #For simple ANN

    for j in range(iterations):

        #Create keras model
        print("Iteration "+str(j+1))
        #file.write("Iteration "+str(j+1)+'\n\n')

        model = get_compiled_model(input_shape, model_type='ann')
        tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)

        #Load data
        min_max_scaler = MinMaxScaler()
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = window_sizes[i]
        tModel.data_handler.sequence_stride = strides[i]
        tModel.data_handler.max_rul = max_ruls[i]

        tModel.data_handler.data_scaler = None
        tModel.data_scaler = min_max_scaler

        tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
        #tModel.data_handler.print_data()
        #tModel.print_data()

        #Train model
        tModel.epochs = 200

        tModel.train_model(verbose=1)
        tModel.evaluate_model(['rhs', 'rmse'], round=2)

        cScores = tModel.scores
        rmse = math.sqrt(cScores['score_1'])
        rmse2 = cScores['rmse']
        rhs = cScores['rhs']
        time = tModel.train_time

        if rmse2 < min_rmse:
            best_model_weights = tModel.model.get_weights()
            #bestModel.set_weights(tModel.model.get_weights())
            bestIndex = j
            min_rmse = rmse2

        tempScoresRMSE[j] = rmse2
        tempScoresRHS[j] = rhs
        tempTime[j] = time

    print(tempScoresRMSE)
    print('Best Score')
    print(min_rmse)
    
    #save best model to file
    #serialize model to JSON

    #print(best_model_weights)
    model = get_compiled_model(input_shape, model_type='ann')
    bestModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps).model
    bestModel.set_weights(best_model_weights)
    model_json = bestModel.to_json()
    with open("bestRULModel_dataset_"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    bestModel.save_weights("bestRULModel_dataset_"+str(i)+".h5")
    print("Saved best model for dataset "+str(i)+" to disk")

Working on dataset 2
Iteration 1
Loading data for dataset 2 with window_size of 17, stride of 1 and maxRUL of 139. Cros-Validation ratio 0
Loading data from file and computing dataframes
training without cv
Epoch 1/200
49599/49599 [==============================] - 1s 15us/step - loss: 5976.6923 - mean_squared_error: 5962.5020
Epoch 2/200
49599/49599 [==============================] - 0s 3us/step - loss: 2358.5296 - mean_squared_error: 2340.2649
Epoch 3/200
49599/49599 [==============================] - 0s 3us/step - loss: 2342.4128 - mean_squared_error: 2324.8242
Epoch 4/200
49599/49599 [==============================] - 0s 3us/step - loss: 2333.9389 - mean_squared_error: 2316.9928
Epoch 5/200
49599/49599 [==============================] - 0s 3us/step - loss: 2324.7207 - mean_squared_error: 2308.3199
Epoch 6/200
49599/49599 [==============================] - 0s 3us/step - loss: 2314.5592 - mean_squared_error: 2298.5985
Epoch 7/200
49599/49599 [==============================] - 0s 3us/

49599/49599 [==============================] - 0s 3us/step - loss: 826.5163 - mean_squared_error: 780.1488
Epoch 68/200
49599/49599 [==============================] - 0s 3us/step - loss: 827.5654 - mean_squared_error: 781.3740
Epoch 69/200
49599/49599 [==============================] - 0s 3us/step - loss: 825.6772 - mean_squared_error: 779.6543
Epoch 70/200
49599/49599 [==============================] - 0s 3us/step - loss: 826.2580 - mean_squared_error: 780.4291
Epoch 71/200
49599/49599 [==============================] - 0s 3us/step - loss: 827.9845 - mean_squared_error: 782.3269
Epoch 72/200
49599/49599 [==============================] - 0s 3us/step - loss: 826.6595 - mean_squared_error: 781.1683
Epoch 73/200
49599/49599 [==============================] - 0s 3us/step - loss: 822.1862 - mean_squared_error: 776.8809
Epoch 74/200
49599/49599 [==============================] - 0s 3us/step - loss: 824.8296 - mean_squared_error: 779.6855
Epoch 75/200
49599/49599 [===========================

49599/49599 [==============================] - 0s 3us/step - loss: 796.7050 - mean_squared_error: 758.7309
Epoch 135/200
49599/49599 [==============================] - 0s 3us/step - loss: 794.6561 - mean_squared_error: 756.7524
Epoch 136/200
49599/49599 [==============================] - 0s 3us/step - loss: 797.5377 - mean_squared_error: 759.7154
Epoch 137/200
49599/49599 [==============================] - 0s 3us/step - loss: 794.1324 - mean_squared_error: 756.3765
Epoch 138/200
49599/49599 [==============================] - 0s 3us/step - loss: 798.4273 - mean_squared_error: 760.7769
Epoch 139/200
49599/49599 [==============================] - 0s 3us/step - loss: 792.5543 - mean_squared_error: 754.9691
Epoch 140/200
49599/49599 [==============================] - 0s 3us/step - loss: 795.0673 - mean_squared_error: 757.5114
Epoch 141/200
49599/49599 [==============================] - 0s 3us/step - loss: 794.2676 - mean_squared_error: 756.7737
Epoch 142/200
49599/49599 [===================

training without cv
Epoch 1/200
49599/49599 [==============================] - 0s 5us/step - loss: 5197.1444 - mean_squared_error: 5183.9657
Epoch 2/200
49599/49599 [==============================] - 0s 3us/step - loss: 2346.7919 - mean_squared_error: 2331.7267
Epoch 3/200
49599/49599 [==============================] - 0s 3us/step - loss: 2337.4181 - mean_squared_error: 2323.1546
Epoch 4/200
49599/49599 [==============================] - 0s 3us/step - loss: 2326.6255 - mean_squared_error: 2312.9549
Epoch 5/200
49599/49599 [==============================] - 0s 3us/step - loss: 2315.2749 - mean_squared_error: 2302.0624
Epoch 6/200
49599/49599 [==============================] - 0s 3us/step - loss: 2302.9098 - mean_squared_error: 2290.0348
Epoch 7/200
49599/49599 [==============================] - 0s 3us/step - loss: 2289.5189 - mean_squared_error: 2276.8852
Epoch 8/200
49599/49599 [==============================] - 0s 3us/step - loss: 2276.2095 - mean_squared_error: 2263.6923
Epoch 9/200


49599/49599 [==============================] - 0s 3us/step - loss: 827.4482 - mean_squared_error: 785.7936
Epoch 69/200
49599/49599 [==============================] - 0s 3us/step - loss: 823.9275 - mean_squared_error: 782.3624
Epoch 70/200
49599/49599 [==============================] - 0s 3us/step - loss: 826.5537 - mean_squared_error: 785.0633
Epoch 71/200
49599/49599 [==============================] - 0s 3us/step - loss: 820.6720 - mean_squared_error: 779.2456
Epoch 72/200
49599/49599 [==============================] - 0s 3us/step - loss: 821.6118 - mean_squared_error: 780.2855
Epoch 73/200
49599/49599 [==============================] - 0s 3us/step - loss: 820.5742 - mean_squared_error: 779.3076
Epoch 74/200
49599/49599 [==============================] - 0s 3us/step - loss: 822.8452 - mean_squared_error: 781.6616
Epoch 75/200
49599/49599 [==============================] - 0s 3us/step - loss: 820.5570 - mean_squared_error: 779.4268
Epoch 76/200
49599/49599 [===========================

...

49599/49599 [==============================] - 0s 3us/step - loss: 850.8734 - mean_squared_error: 805.9681
Epoch 74/200
49599/49599 [==============================] - 0s 3us/step - loss: 851.1339 - mean_squared_error: 806.3444
Epoch 75/200
49599/49599 [==============================] - 0s 3us/step - loss: 852.4136 - mean_squared_error: 807.7467
Epoch 76/200
49599/49599 [==============================] - 0s 3us/step - loss: 852.6248 - mean_squared_error: 808.0774
Epoch 77/200
49599/49599 [==============================] - 0s 3us/step - loss: 848.7833 - mean_squared_error: 804.3350
Epoch 78/200
49599/49599 [==============================] - 0s 3us/step - loss: 851.0007 - mean_squared_error: 806.6651
Epoch 79/200
49599/49599 [==============================] - 0s 3us/step - loss: 849.9106 - mean_squared_error: 805.6939
Epoch 80/200
49599/49599 [==============================] - 0s 3us/step - loss: 847.6009 - mean_squared_error: 803.5067
Epoch 81/200
49599/49599 [===========================

49599/49599 [==============================] - 0s 3us/step - loss: 818.2889 - mean_squared_error: 779.4802
Epoch 142/200
49599/49599 [==============================] - 0s 3us/step - loss: 816.3352 - mean_squared_error: 777.5408
Epoch 143/200
49599/49599 [==============================] - 0s 3us/step - loss: 809.2459 - mean_squared_error: 770.4581
Epoch 144/200
49599/49599 [==============================] - 0s 3us/step - loss: 811.7675 - mean_squared_error: 773.0039
Epoch 145/200
49599/49599 [==============================] - 0s 3us/step - loss: 812.5377 - mean_squared_error: 773.7959
Epoch 146/200
49599/49599 [==============================] - 0s 3us/step - loss: 810.9173 - mean_squared_error: 772.1921
Epoch 147/200
49599/49599 [==============================] - 0s 3us/step - loss: 810.6534 - mean_squared_error: 771.9695
Epoch 148/200
49599/49599 [==============================] - 0s 3us/step - loss: 811.0808 - mean_squared_error: 772.4227
Epoch 149/200
49599/49599 [===================

49599/49599 [==============================] - 0s 3us/step - loss: 2297.8906 - mean_squared_error: 2283.1017
Epoch 7/200
49599/49599 [==============================] - 0s 3us/step - loss: 2276.9601 - mean_squared_error: 2262.0722
Epoch 8/200
49599/49599 [==============================] - 0s 3us/step - loss: 2244.1691 - mean_squared_error: 2228.6229
Epoch 9/200
49599/49599 [==============================] - 0s 3us/step - loss: 2196.0869 - mean_squared_error: 2179.1514
Epoch 10/200
49599/49599 [==============================] - 0s 3us/step - loss: 2134.6287 - mean_squared_error: 2115.8267
Epoch 11/200
49599/49599 [==============================] - 0s 3us/step - loss: 2043.4316 - mean_squared_error: 2021.8975
Epoch 12/200
49599/49599 [==============================] - 0s 3us/step - loss: 1912.3595 - mean_squared_error: 1887.0590
Epoch 13/200
49599/49599 [==============================] - 0s 3us/step - loss: 1748.6830 - mean_squared_error: 1719.0250
Epoch 14/200
49599/49599 [==============

49599/49599 [==============================] - 0s 3us/step - loss: 824.2377 - mean_squared_error: 778.0072
Epoch 75/200
49599/49599 [==============================] - 0s 3us/step - loss: 823.7693 - mean_squared_error: 777.7485
Epoch 76/200
49599/49599 [==============================] - 0s 3us/step - loss: 827.2564 - mean_squared_error: 781.4466
Epoch 77/200
49599/49599 [==============================] - 0s 3us/step - loss: 826.6937 - mean_squared_error: 781.1029
Epoch 78/200
49599/49599 [==============================] - 0s 3us/step - loss: 824.8852 - mean_squared_error: 779.4638
Epoch 79/200
49599/49599 [==============================] - 0s 3us/step - loss: 822.5577 - mean_squared_error: 777.2960
Epoch 80/200
49599/49599 [==============================] - 0s 3us/step - loss: 823.0290 - mean_squared_error: 777.9415
Epoch 81/200
49599/49599 [==============================] - 0s 3us/step - loss: 821.8423 - mean_squared_error: 776.9270
Epoch 82/200
49599/49599 [===========================

49599/49599 [==============================] - 0s 3us/step - loss: 791.2880 - mean_squared_error: 752.7837
Epoch 143/200
49599/49599 [==============================] - 0s 3us/step - loss: 791.5937 - mean_squared_error: 753.1561
Epoch 144/200
49599/49599 [==============================] - 0s 3us/step - loss: 797.2571 - mean_squared_error: 758.8620
Epoch 145/200
49599/49599 [==============================] - 0s 3us/step - loss: 787.9923 - mean_squared_error: 749.6556
Epoch 146/200
49599/49599 [==============================] - 0s 3us/step - loss: 792.9146 - mean_squared_error: 754.6392
Epoch 147/200
49599/49599 [==============================] - 0s 3us/step - loss: 791.9274 - mean_squared_error: 753.7116
Epoch 148/200
49599/49599 [==============================] - 0s 3us/step - loss: 786.6081 - mean_squared_error: 748.4410
Epoch 149/200
49599/49599 [==============================] - 0s 3us/step - loss: 790.2755 - mean_squared_error: 752.1420
Epoch 150/200
49599/49599 [===================

In [6]:
# load json and create model
json_file = open('bestRULModel_dataset_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("bestRULModel_dataset_1.h5")
print("Loaded model from disk")


#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Create and compile the models
#nFeatures = len(selected_features)
#shapeSN = nFeatures*windowSize
#modelRULSN = RULmodel_SN_1(shapeSN)
loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
min_max_scaler = MinMaxScaler()

tModel = SequenceTunableModelRegression('ModelRUL_SN_1', loaded_model, lib_type='keras', data_handler=dHandler_cmaps)

i = 1

tModel.data_handler.change_dataset(i)
tModel.data_handler.sequence_length = window_sizes[i]
tModel.data_handler.sequence_stride = strides[i]
tModel.data_handler.max_rul = max_ruls[i]

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.print_data()
#print(tModel.model.get_weights())
tModel.get_model_description()



Loaded model from disk
Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes
Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 20)                6740      
_________________________________________________________________
fc2 (Dense)                  (None, 20)                420       
_________________________________________________________________
out (Dense)                  (None, 1)                 21        
Total params: 7,181
Trainable params: 7,181
Non-trainable params: 0
_________________________________________________________________


In [7]:
# evaluate loaded model on test data
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
print(cScores)
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 315us/step
scores
{'loss': 270.6421185302734, 'score_1': 206.36581756591798, 'rhs': 3.4385813531834817, 'rmse': 14.358272876638054}
{'loss': 270.6421185302734, 'score_1': 206.36581756591798, 'rhs': 3.4385813531834817, 'rmse': 14.358272876638054}
RMSE: 14.36543830051551
RMSE2: 14.358272876638054
RHS: 3.4385813531834817
Time : None seconds
